### 🔍 1. Setup e Leitura dos Dados
---

O dataset `MQD-1465` contém 1465 frases coletadas da plataforma "Meu Querido Diário" (www.meuqueridodiario.com.br)

INPUTS NECESSÁRIOS
* data/MQD-1465.csv (dataset original)
* data/logs_brutos (pasta com os logs brutos do experimento no PCIBex Farm)

TRATAMENTO PROPOSTO</br>
* Divisão do dataset em 10 blocos com no máximo 150 frases</br>
* Remoção de frases duplicadas no dataset original</br>
* Fixaçao de seed para reprodutibilidade




In [ ]:
#Importações de bibliotecas

import pandas as pd
import numpy as np
import os
import csv
from pathlib import Path

import pandas as pd
import random
import sys
import io

import warnings
warnings.filterwarnings('ignore')


In [5]:
# ============================================================================
# CONFIGURACAO
# ============================================================================
SEED = 42
INPUT_FILE = 'data/MQD-1465.csv'

print("="*80)
print("RANDOMIZACAO MQD-1465 (Seed: 42)")
print("="*80)

# ============================================================================
# ETAPA 1: CARREGAR E RANDOMIZAR
# ============================================================================
print("\n[1] Carregando e randomizando dados...")

df = pd.read_csv(INPUT_FILE, encoding='utf-8')
df.columns = ['id', 'frase', 'classificacao', 'juizes']
print(f"    {len(df)} registros carregados")

df_random = df.sample(frac=1, random_state=SEED).reset_index(drop=True)
print(f"    Randomizado com seed {SEED}")
print(f"    Primeiros IDs: {df_random['id'].head(5).tolist()}")

# ============================================================================
# ETAPA 2: REMOVER DUPLICATAS
# ============================================================================
print("\n[2] Removendo duplicatas...")

duplicatas = df_random[df_random.duplicated(subset=['frase'], keep=False)]
if len(duplicatas) > 0:
    print(f"    Duplicatas encontradas:")
    for frase, grupo in duplicatas.groupby('frase'):
        ids = grupo['id'].tolist()
        mantido = df_random[df_random['frase'] == frase]['id'].iloc[0]
        removidos = [i for i in ids if i != mantido]
        print(f"    - IDs {ids}: mantendo {mantido}, removendo {removidos}")

df_final = df_random.drop_duplicates(subset=['frase'], keep='first').reset_index(drop=True)
print(f"    Total apos remocao: {len(df_final)} registros")

# ============================================================================
# ETAPA 3: PADRONIZAR FRASES
# ============================================================================
print("\n[3] Padronizando frases...")

df_final['frase'] = df_final['frase'].str.strip().str.replace('"', '', regex=False)
print(f"    Espacos e aspas removidos")

# ============================================================================
# ETAPA 4: SALVAR ARQUIVO
# ============================================================================
# Define nome do arquivo dinamicamente baseado no numero de registros
num_registros = len(df_final)
OUTPUT_FILE = f'data/logs_processados/MQD-{num_registros}_random_sem_repeticao.csv'

print(f"\n[4] Salvando '{OUTPUT_FILE}'...")

# Garante que a pasta de destino exista
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

with open(OUTPUT_FILE, 'w', encoding='utf-8', newline='') as f:
    f.write('id\tfrase\tclassificacao\tjuizes\n')
    for _, row in df_final.iterrows():
        f.write(f'{row["id"]}\t"{row["frase"]}"\t{row["classificacao"]}\t{row["juizes"]}\n')

print(f"    Arquivo salvo com sucesso!")


# ============================================================================
# ESTATISTICAS
# ============================================================================
print("\n" + "="*80)
print("ESTATISTICAS")
print("="*80)

print(f"\nTotal: {len(df_final)} registros\n")

print("Classificacao:")
for c in sorted(df_final['classificacao'].unique()):
    n = (df_final['classificacao'] == c).sum()
    print(f"  {c:>2}: {n:>4} ({100*n/len(df_final):>5.1f}%)")

print("\nJuizes:")
for j in sorted(df_final['juizes'].unique()):
    n = (df_final['juizes'] == j).sum()
    print(f"  {j}: {n:>4} ({100*n/len(df_final):>5.1f}%)")

print("\n" + "="*80)
print("CONCLUIDO!")
print("="*80)
print(f"\nArquivo: {OUTPUT_FILE} ({len(df_final)} registros)")
print(f"DataFrame 'df_final' disponivel para uso")
print("\nExemplos de uso:")
print("  df_final.head()          # Ver primeiras linhas")
print("  df_final.info()          # Informacoes do dataset")
print("  df_final.describe()      # Estatisticas")

# Criar variavel global para uso no notebook
df_mqd = df_final.copy()

RANDOMIZACAO MQD-1465 (Seed: 42)

[1] Carregando e randomizando dados...
    1465 registros carregados
    Randomizado com seed 42
    Primeiros IDs: [1039, 213, 314, 598, 931]

[2] Removendo duplicatas...
    Duplicatas encontradas:
    - IDs [475, 476]: mantendo 475, removendo [476]
    - IDs [952, 953]: mantendo 952, removendo [953]
    Total apos remocao: 1463 registros

[3] Padronizando frases...
    Espacos e aspas removidos

[4] Salvando 'data/logs_processados/MQD-1463_random_sem_repeticao.csv'...
    Arquivo salvo com sucesso!

ESTATISTICAS

Total: 1463 registros

Classificacao:
  -1:  544 ( 37.2%)
   0:  409 ( 28.0%)
   1:  510 ( 34.9%)

Juizes:
  2:  525 ( 35.9%)
  3:  936 ( 64.0%)
  4:    2 (  0.1%)

CONCLUIDO!

Arquivo: data/logs_processados/MQD-1463_random_sem_repeticao.csv (1463 registros)
DataFrame 'df_final' disponivel para uso

Exemplos de uso:
  df_final.head()          # Ver primeiras linhas
  df_final.info()          # Informacoes do dataset
  df_final.describe()   

In [6]:

def processar_arquivo_randomizado():
    """
    Processa o arquivo randomizado aplicando as seguintes transformações:
    1. Renomeia a coluna id para id_original
    2. Cria novo id sequencial
    3. Adiciona coluna de bloco (incremento a cada 150 registros)
    """
    # Encontra o arquivo MQD-*_random_sem_repeticao.csv automaticamente
    import glob
    arquivos = glob.glob("data/logs_processados/MQD-*_random_sem_repeticao.csv")
    
    if not arquivos:
        raise FileNotFoundError("Nenhum arquivo MQD-*_random_sem_repeticao.csv encontrado!")
    
    if len(arquivos) > 1:
        print(f"AVISO: Múltiplos arquivos encontrados. Usando o mais recente: {arquivos}")
        # Ordena por tempo de modificação e pega o mais recente
        arquivos.sort(key=os.path.getmtime, reverse=True)
    
    input_path = arquivos[0]
    print(f"Processando arquivo: {input_path}")
    
    # Carrega arquivo usando tab como separador e protegendo aspas
    df = pd.read_csv(
        input_path,
        sep='\t',
        usecols=['id', 'frase'],
        quoting=csv.QUOTE_ALL,
        quotechar='"',
        encoding='utf-8'
    )
    
    # Renomear coluna id para id_original
    df = df.rename(columns={'id': 'id_original'})
    
    # Criar novo id sequencial começando de 1
    df.insert(0, 'id', range(1, len(df) + 1))
    
    # Criar coluna de bloco (incremento a cada 150 registros)
    df['bloco'] = (df.index // 150) + 1
    
    # Define nome do arquivo de saida dinamicamente baseado no numero de registros
    num_registros = len(df)
    output_path = f"data/logs_processados/MQD-{num_registros}_blocos.csv"
    
    # Salvar novo arquivo mantendo as aspas e usando tab como separador
    df.to_csv(
        output_path,
        sep='\t',
        index=False,
        quoting=csv.QUOTE_ALL,
        quotechar='"'
    )
    
    print(f"Arquivo processado e salvo em: {output_path}")
    print(f"Total de registros: {len(df)}")
    print(f"Total de blocos: {df['bloco'].max()}")
    
    print("\nPrimeiras linhas do arquivo processado:")
    print(df.head())
    
    return df

# Executar processamento
df_blocos_randomizados = processar_arquivo_randomizado()

Processando arquivo: data/logs_processados\MQD-1463_random_sem_repeticao.csv
Arquivo processado e salvo em: data/logs_processados/MQD-1463_blocos.csv
Total de registros: 1463
Total de blocos: 10

Primeiras linhas do arquivo processado:
   id  id_original                                              frase  bloco
0   1         1039  Você sabia que o menino que mais vai te dar va...      1
1   2          213  Hoje com 21 anos, como que por auxilio lá do a...      1
2   3          314  Nesse momento to evitando ela, e ta me dando u...      1
3   4          598  Meus sentidos de garota diziam que tinha algum...      1
4   5          931  Aquela promessa que eu te fiz naquele natal, e...      1


### 2. Tratamento dos logs do PCIbex
---
* Criação dos blocos concatenados com todas as anotações
* Tamanho variável de blocos (anotadores x frases)
* Dataset com ParticipantMD5, GeneroCod, frase, Value e duracao

In [8]:
def carregar_blocos_randomizados():
    """
    Carrega o arquivo com as frases e seus respectivos blocos
    """
    # Encontra o arquivo MQD-*_blocos.csv automaticamente
    import glob
    arquivos = glob.glob("data/logs_processados/MQD-*_blocos.csv")
    
    if not arquivos:
        raise FileNotFoundError("Nenhum arquivo MQD-*_blocos.csv encontrado!")
    
    if len(arquivos) > 1:
        # Ordena por tempo de modificação e pega o mais recente
        arquivos.sort(key=os.path.getmtime, reverse=True)
    
    blocos_path = arquivos[0]
    print(f"Carregando arquivo: {blocos_path}")
    
    return pd.read_csv(
        blocos_path,
        sep='\t',
        quoting=csv.QUOTE_ALL,
        quotechar='"',
        encoding='utf-8'
    )

def extrair_genero(df):
    """
    Extrai o gênero dos participantes do arquivo de log
    """
    genero_data = df[
        (df["Label"] == "genero") &
        (df["PennElementName"] == "selecionaGenero") &
        (df["Parameter"] == "Selected")
    ][["ParticipantMD5", "Value"]].copy()
    
    genero_data["GeneroCod"] = genero_data["Value"].str.lower().map(
        lambda g: "m" if "masculino" in g else "f"
    )
    
    return genero_data[["ParticipantMD5", "GeneroCod"]]

def processar_arquivo_log(file_path, frases_bloco, num_bloco):
    """
    Processa um arquivo de log substituindo as frases originais pelas do bloco correto
    e calcula a duração dos trials
    """
    try:
        # Lê o arquivo de log pulando as 19 linhas de cabeçalho
        df = pd.read_csv(file_path, skiprows=19, header=None)
        
        # Define as colunas apenas uma vez
        df.columns = [
            "ReceptionTime", "ParticipantMD5", "Controller", "ItemNumber", 
            "InnerElementNumber", "Label", "Group", "PennElementType", 
            "PennElementName", "Parameter", "Value", "EventTime", "Comments"
        ]
        
        # Converte EventTime para numérico
        df["EventTime"] = pd.to_numeric(df["EventTime"], errors="coerce")
        
        # Extrai informações de gênero
        generos = extrair_genero(df)
        
        # Filtra eventos de início e fim dos trials
        df_trials = df[
            (df["Label"] == "frases") & 
            (df["Value"].isin(["Start", "End"]))
        ].copy()
        
        # Ordena por participante, item e tempo
        df_trials = df_trials.sort_values(["ParticipantMD5", "ItemNumber", "EventTime"])
        
        # Calcula duração entre Start e End para cada trial
        duracoes = []
        for (participant, item), group in df_trials.groupby(["ParticipantMD5", "ItemNumber"]):
            if len(group) == 2:  # Verifica se tem tanto Start quanto End
                start_time = group[group["Value"] == "Start"]["EventTime"].iloc[0]
                end_time = group[group["Value"] == "End"]["EventTime"].iloc[0]
                duracao = (end_time - start_time) / 1000.0  # Converte para segundos
                
                # Ajusta o ItemNumber da mesma forma que é feito para as classificações
                adjusted_item = item - 3 if num_bloco == 1 else item - 3 + ((num_bloco - 1) * 150)
                
                duracoes.append({
                    "ParticipantMD5": participant,
                    "ItemNumber": adjusted_item,
                    "duracao": duracao
                })
        
        df_duracoes = pd.DataFrame(duracoes)
        
        # Filtra apenas as linhas de classificação
        df_classificacoes = df[
            (df["Label"] == "frases") & 
            (df["Parameter"] == "Selection")
        ].copy()
        
        # Ajusta o ItemNumber baseado no número do bloco
        if num_bloco == 1:
            df_classificacoes["ItemNumber"] = df_classificacoes["ItemNumber"] - 3
        else:
            df_classificacoes["ItemNumber"] = df_classificacoes["ItemNumber"] - 3 + ((num_bloco - 1) * 150)
        
        # Merge com as frases do bloco
        df_final = df_classificacoes.merge(
            frases_bloco[["id", "frase"]],
            left_on="ItemNumber",
            right_on="id",
            how="inner"
        )
        
        # Adiciona informação de gênero
        df_final = df_final.merge(
            generos,
            on="ParticipantMD5",
            how="left"
        )
        
        # Adiciona informação de duração
        df_final = df_final.merge(
            df_duracoes,
            on=["ParticipantMD5", "ItemNumber"],
            how="left"
        )
        
        return df_final[["ParticipantMD5", "GeneroCod", "frase", "Value", "duracao"]]
        
    except Exception as e:
        print(f"Erro ao processar arquivo: {str(e)}")
        print(f"Local do erro: {e.__traceback__.tb_lineno}")
        raise
def processar_todos_arquivos():
    """
    Processa todos os arquivos de log, correlacionando com os blocos corretos
    """
    #pasta_destino = criar_pasta_tratamento()
    pasta_destino = "data/logs_processados"
    os.makedirs(pasta_destino, exist_ok=True)
    df_blocos = carregar_blocos_randomizados()
    
    # Para cada arquivo de log (0 a 9)
    for i in range(10):
        num_bloco = i + 1
        if i == 0:
            arquivo_origem = "data/logs_brutos/results_prod.csv"
        else:
            arquivo_origem = f"data/logs_brutos/results_prod ({i}).csv"
            
        arquivo_destino = f"{pasta_destino}/bloco_{num_bloco}_concatenado.csv"
        
        if os.path.exists(arquivo_origem):
            print(f"\nProcessando bloco {num_bloco} - Arquivo: {arquivo_origem}")
            
            frases_bloco = df_blocos[df_blocos['bloco'] == num_bloco].copy()
            print(f"Frases encontradas no bloco {num_bloco}: {len(frases_bloco)}")
            
            try:
                df_processado = processar_arquivo_log(arquivo_origem, frases_bloco, num_bloco)
                
                if len(df_processado) > 0:
                    df_processado.to_csv(
                        arquivo_destino,
                        sep='\t',
                        index=False,
                        quoting=csv.QUOTE_ALL,
                        quotechar='"',
                        encoding='utf-8',
                        float_format='%.3f'  # Format float numbers with 3 decimal places
                    )
                    print(f"Arquivo salvo: {arquivo_destino}")
                    print(f"Total de classificações: {len(df_processado)}")
                else:
                    print(f"AVISO: Nenhuma classificação encontrada para o bloco {num_bloco}")
            except Exception as e:
                print(f"Erro ao processar bloco {num_bloco}: {str(e)}")
        else:
            print(f"Arquivo não encontrado: {arquivo_origem}")

# Executa o processamento
processar_todos_arquivos()

Carregando arquivo: data/logs_processados\MQD-1463_blocos.csv

Processando bloco 1 - Arquivo: data/logs_brutos/results_prod.csv
Frases encontradas no bloco 1: 150
Arquivo salvo: data/logs_processados/bloco_1_concatenado.csv
Total de classificações: 1350

Processando bloco 2 - Arquivo: data/logs_brutos/results_prod (1).csv
Frases encontradas no bloco 2: 150
Arquivo salvo: data/logs_processados/bloco_2_concatenado.csv
Total de classificações: 1200

Processando bloco 3 - Arquivo: data/logs_brutos/results_prod (2).csv
Frases encontradas no bloco 3: 150
Arquivo salvo: data/logs_processados/bloco_3_concatenado.csv
Total de classificações: 1800

Processando bloco 4 - Arquivo: data/logs_brutos/results_prod (3).csv
Frases encontradas no bloco 4: 150
Arquivo salvo: data/logs_processados/bloco_4_concatenado.csv
Total de classificações: 1200

Processando bloco 5 - Arquivo: data/logs_brutos/results_prod (4).csv
Frases encontradas no bloco 5: 150
Arquivo salvo: data/logs_processados/bloco_5_concaten

### 

In [9]:
# Criacao do dataset com todas as anotaçoes

# Define o caminho da pasta
logs_path = Path('data/logs_processados')

# Lista todos os arquivos de bloco concatenados
bloco_files = list(logs_path.glob('bloco_*_concatenado.csv'))

# Cria uma lista para armazenar os DataFrames
dfs = []

# Lê cada arquivo mantendo a estrutura original
for file in bloco_files:
    df = pd.read_csv(file, sep='\t')  # Remove a definição manual de colunas
    dfs.append(df)

# Concatena todos os DataFrames
df_totais_com_duracao = pd.concat(dfs, ignore_index=True)

# Remove duplicatas se houver
#df_totais_com_duracao = df_totais_com_duracao.drop_duplicates()

# Salva o DataFrame consolidado mantendo o formato original
output_file = logs_path / 'MQD-13317_anotacoes_totais.csv'
df_totais_com_duracao.to_csv(output_file, sep='\t', index=False)  # Adiciona sep='\t'

print(f"Arquivo criado com sucesso em: {output_file}")
print(f"Total de registros: {len(df_totais_com_duracao)}")

Arquivo criado com sucesso em: data\logs_processados\MQD-13317_anotacoes_totais.csv
Total de registros: 13317


In [10]:
# Separa por gênero

# Lê o arquivo original
df = pd.read_csv('data/logs_processados/MQD-13317_anotacoes_totais.csv', sep='\t')

# Separa por gênero
df_totais_masculinos = df[df['GeneroCod'] == 'm']
df_totais_femininos = df[df['GeneroCod'] == 'f']

# Salva os arquivos separados
output_masculino = logs_path / 'MQD-7015_anotacoes_totais_masculinas.csv'
output_feminino = logs_path / 'MQD-6302_anotacoes_totais_femininas.csv'

df_totais_masculinos.to_csv(output_masculino, sep='\t', index=False)
df_totais_femininos.to_csv(output_feminino, sep='\t', index=False)

print(f"Dataset masculino salvo em: {output_masculino}")
print(f"Total de registros masculinos: {len(df_totais_masculinos)}")
print(f"\nDataset feminino salvo em: {output_feminino}")
print(f"Total de registros femininos: {len(df_totais_femininos)}")

Dataset masculino salvo em: data\logs_processados\MQD-7015_anotacoes_totais_masculinas.csv
Total de registros masculinos: 7015

Dataset feminino salvo em: data\logs_processados\MQD-6302_anotacoes_totais_femininas.csv
Total de registros femininos: 6302


## Geração do Dataset Final: 4 Classificações com Maioria Estrita

Esta seção gera o dataset final garantindo:
1. **Exatamente 4 classificações** por gênero para cada frase
2. **Maioria estrita** definida (sem empate 2-2)
3. **Paridade** entre gêneros (apenas frases presentes em ambos)

**Lógica de seleção:**
- Usar as 4 primeiras anotações (ordem temporal)
- Se houver empate 2-2, substituir última anotação por próxima disponível
- Se impossível resolver empate, descartar a frase

**Maioria estrita válida:**
- ✓ 4-0-0 (unanimidade)
- ✓ 3-1-0 
- ✓ 2-1-1 (uma classe à frente)
- ✗ 2-2-0 (empate - descartada)

In [11]:
# =============================================================================
# GERAÇÃO DO DATASET FINAL: 4 CLASSIFICAÇÕES COM MAIORIA ESTRITA
# =============================================================================

# Carregar dados brutos de anotações individuais
df_masc_raw = pd.read_csv(logs_path / 'MQD-7015_anotacoes_totais_masculinas.csv', sep='\t')
df_fem_raw = pd.read_csv(logs_path / 'MQD-6302_anotacoes_totais_femininas.csv', sep='\t')

# Adicionar índice original para preservar ordem temporal
df_masc_raw['ordem_original'] = df_masc_raw.index
df_fem_raw['ordem_original'] = df_fem_raw.index

print("="*80)
print("GERAÇÃO DO DATASET FINAL")
print("="*80)
print(f"Anotações masculinas: {len(df_masc_raw)}")
print(f"Anotações femininas: {len(df_fem_raw)}")


def tem_maioria_estrita(contagem):
    """
    Verifica se há maioria estrita (sem empate no topo).
    
    Maioria estrita = UMA classe com mais votos que qualquer outra.
    - 4-0-0: ✓ (unanimidade)
    - 3-1-0: ✓ 
    - 2-1-1: ✓ (uma classe à frente)
    - 2-2-0: ✗ (empate)
    """
    valores = sorted(contagem.values, reverse=True)
    if len(valores) >= 2 and valores[0] == valores[1]:
        return False
    return True


def selecionar_4_com_maioria(grupo):
    """
    Seleciona exatamente 4 anotações que garantam maioria estrita.
    
    Estratégia:
    1. Ordenar por ordem temporal
    2. Tentar com as 4 primeiras
    3. Se empate (2-2), substituir última por próxima disponível
    """
    grupo = grupo.sort_values('ordem_original').reset_index(drop=True)
    n = len(grupo)
    
    if n < 4:
        return None
    
    # Tentar com as 4 primeiras
    primeiras_4 = grupo.head(4)
    contagem = primeiras_4['Value'].value_counts()
    
    if tem_maioria_estrita(contagem):
        return primeiras_4
    
    # Se tem empate e mais anotações, tentar substituições
    if n > 4:
        for i in range(4, n):
            candidatos_idx = [0, 1, 2, i]
            selecao = grupo.iloc[candidatos_idx]
            contagem = selecao['Value'].value_counts()
            if tem_maioria_estrita(contagem):
                return selecao
        
        for i in range(4, n):
            candidatos_idx = [0, 1, i, 3]
            selecao = grupo.iloc[candidatos_idx]
            contagem = selecao['Value'].value_counts()
            if tem_maioria_estrita(contagem):
                return selecao
        
        for i in range(4, n):
            for j in range(i+1, n) if n > i+1 else []:
                candidatos_idx = [0, 1, i, j]
                selecao = grupo.iloc[candidatos_idx]
                contagem = selecao['Value'].value_counts()
                if tem_maioria_estrita(contagem):
                    return selecao
    
    return None


def processar_grupo(df, genero):
    """Processa todas as frases de um gênero."""
    resultados = []
    descartadas = []
    
    for frase in df['frase'].unique():
        grupo = df[df['frase'] == frase]
        selecao = selecionar_4_com_maioria(grupo)
        
        if selecao is not None:
            contagem = selecao['Value'].value_counts()
            maioria = contagem.idxmax()
            votos_maioria = contagem.max()
            
            votos_positiva = (selecao['Value'] == 'positiva').sum()
            votos_negativa = (selecao['Value'] == 'negativa').sum()
            votos_neutra = (selecao['Value'] == 'neutra').sum()
            duracao_media = selecao['duracao'].mean()
            
            resultados.append({
                'frase': frase,
                f'total_classificacoes_{genero}': 4,
                f'classificacao_majoritaria_{genero}': maioria,
                f'votos_maioria_{genero}': votos_maioria,
                f'total_positiva_{genero}': votos_positiva,
                f'total_negativa_{genero}': votos_negativa,
                f'total_neutra_{genero}': votos_neutra,
                f'duracao_media_{genero}': duracao_media
            })
        else:
            descartadas.append(frase)
    
    return pd.DataFrame(resultados), descartadas


# Processar cada gênero
print("\n[1] Processando anotações masculinas...")
df_result_masc, descartadas_masc = processar_grupo(df_masc_raw, 'masculino')
print(f"    ✓ {len(df_result_masc)} frases com maioria estrita")
print(f"    ✗ {len(descartadas_masc)} frases descartadas (empate irresolvível)")

print("\n[2] Processando anotações femininas...")
df_result_fem, descartadas_fem = processar_grupo(df_fem_raw, 'feminino')
print(f"    ✓ {len(df_result_fem)} frases com maioria estrita")
print(f"    ✗ {len(descartadas_fem)} frases descartadas (empate irresolvível)")

# Combinar (inner join - apenas frases em comum)
print("\n[3] Combinando datasets...")
df_final = df_result_masc.merge(df_result_fem, on='frase', how='inner')
print(f"    ✓ {len(df_final)} frases em comum (ambos gêneros com maioria estrita)")

# Calcular concordância entre grupos
df_final['concordancia_grupos'] = (
    df_final['classificacao_majoritaria_masculino'] == 
    df_final['classificacao_majoritaria_feminino']
).astype(int)

n_concordam = df_final['concordancia_grupos'].sum()
n_discordam = len(df_final) - n_concordam

print(f"\n[4] Concordância entre grupos:")
print(f"    Concordam: {n_concordam} ({100*n_concordam/len(df_final):.1f}%)")
print(f"    Discordam: {n_discordam} ({100*n_discordam/len(df_final):.1f}%)")

# Ordenar colunas
colunas_ordem = [
    'frase',
    'duracao_media_masculino', 'total_classificacoes_masculino',
    'classificacao_majoritaria_masculino', 'votos_maioria_masculino',
    'total_positiva_masculino', 'total_negativa_masculino', 'total_neutra_masculino',
    'duracao_media_feminino', 'total_classificacoes_feminino',
    'classificacao_majoritaria_feminino', 'votos_maioria_feminino',
    'total_positiva_feminino', 'total_negativa_feminino', 'total_neutra_feminino',
    'concordancia_grupos'
]
df_final = df_final[colunas_ordem]

# Salvar
output_file = logs_path / f'MQD-{len(df_final)}_majoritarias.csv'
df_final.to_csv(output_file, sep='\t', index=False)
print(f"\n[5] Arquivo salvo: {output_file.name}")

print("\n" + "="*80)
print("ESTATÍSTICAS DO DATASET FINAL")
print("="*80)

print("\nDistribuição de votos na maioria:")
print("\nMasculino:")
print(df_final['votos_maioria_masculino'].value_counts().sort_index())
print("\nFeminino:")
print(df_final['votos_maioria_feminino'].value_counts().sort_index())

print("\nDistribuição de classes majoritárias:")
print("\nMasculino:")
print(df_final['classificacao_majoritaria_masculino'].value_counts())
print("\nFeminino:")
print(df_final['classificacao_majoritaria_feminino'].value_counts())

print("\n" + "="*80)
print("✓ DATASET FINAL GERADO COM SUCESSO")
print("="*80)

GERAÇÃO DO DATASET FINAL
Anotações masculinas: 7015
Anotações femininas: 6302

[1] Processando anotações masculinas...
    ✓ 1388 frases com maioria estrita
    ✗ 75 frases descartadas (empate irresolvível)

[2] Processando anotações femininas...
    ✓ 1263 frases com maioria estrita
    ✗ 200 frases descartadas (empate irresolvível)

[3] Combinando datasets...
    ✓ 1209 frases em comum (ambos gêneros com maioria estrita)

[4] Concordância entre grupos:
    Concordam: 1021 (84.4%)
    Discordam: 188 (15.6%)

[5] Arquivo salvo: MQD-1209_majoritarias.csv

ESTATÍSTICAS DO DATASET FINAL

Distribuição de votos na maioria:

Masculino:
votos_maioria_masculino
2    129
3    460
4    620
Name: count, dtype: int64

Feminino:
votos_maioria_feminino
2    115
3    598
4    496
Name: count, dtype: int64

Distribuição de classes majoritárias:

Masculino:
classificacao_majoritaria_masculino
positiva    475
negativa    444
neutra      290
Name: count, dtype: int64

Feminino:
classificacao_majoritaria_